In [ ]:
#! pip3 install openpyxl
#! pip3 install sentence_transformers

In [2]:
import pandas as pd
import numpy as np
import psycopg2
import requests
import re
import time
from sentence_transformers import SentenceTransformer


/home/user03/Documents/SVN/UrFU.term3/Hackaton2023/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Справочник Вузов (первая закладка)

In [3]:
edu_df = pd.read_excel('Вузы РФ (от 07.12.23).xlsx')

In [39]:
edu_df

,IsFederal,Id,StatusName,StatusCode,TypeName,TypeCode,RegionName,RegionCode,FederalDistrictName,FederalDistrictShortName,...,StartedSupplementIssueDate,StoppedSupplementId,StoppedSupplementNumber,StoppedSupplementSerialNumber,StoppedSupplementFormNumber,StoppedSupplementIssueDate,OldRegNumber,OldSerialNumber,OldFormNumber,OldIssueDate
0,NaN,000000de-9b01-0511-a861-69d27006612a,Недействующее,NaN,Не определен,Unknown,Тверская область,69.0,Центральный федеральный округ,ЦФО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,000000de-9b01-0511-a861-69d27006612a,Недействующее,NaN,Не определен,Unknown,Тверская область,69.0,Центральный федеральный округ,ЦФО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,000000de-9b01-0511-a861-69d27006612a,Недействующее,NaN,Не определен,Unknown,Тверская область,69.0,Центральный федеральный округ,ЦФО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,000000de-9b01-0511-a861-69d27006612a,Недействующее,NaN,Не определен,Unknown,Тверская область,69.0,Центральный федеральный округ,ЦФО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0000ff25-0c15-41a8-ad87-17a739919454,Действующее,InForce,Не определен,Unknown,Свердловская область,66.0,Уральский федеральный округ,УФО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692566,1.0,ffffdaa5-0b87-4774-bc2a-f660ae77c66e,Недействующее,NaN,Постоянное,Permanent,Нижегородская область,52.0,Приволжский федеральный округ,ПФО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692567,1.0,ffffdaa5-0b87-4774-bc2a-f660ae77c66e,Недействующее,NaN,Постоянное,Permanent,Нижегородская область,52.0,Приволжский федеральный округ,ПФО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692568,1.0,ffffdaa5-0b87-4774-bc2a-f660ae77c66e,Недействующее,NaN,Постоянное,Permanent,Нижегородская область,52.0,Приволжский федеральный округ,ПФО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692569,1.0,ffffdaa5-0b87-4774-bc2a-f660ae77c66e,Недействующее,NaN,Постоянное,Permanent,Нижегородская область,52.0,Приволжский федеральный округ,ПФО,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
edu_df.info(max_cols=edu_df.shape[1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692571 entries, 0 to 692570
Data columns (total 138 columns):
 #    Column                            Non-Null Count   Dtype  
---   ------                            --------------   -----  
 0    IsFederal                         357471 non-null  float64
 1    Id                                692571 non-null  object 
 2    StatusName                        692571 non-null  object 
 3    StatusCode                        245633 non-null  object 
 4    TypeName                          692571 non-null  object 
 5    TypeCode                          692571 non-null  object 
 6    RegionName                        688239 non-null  object 
 7    RegionCode                        688239 non-null  float64
 8    FederalDistrictName               686951 non-null  object 
 9    FederalDistrictShortName          686951 non-null  object 
 10   FederalDistrictCode               686951 non-null  object 
 11   RegNumber                         692

In [213]:
edu_columns_df = edu_df.agg(['count','nunique']).T.reset_index()
edu_columns_df

,index,count,nunique
0,IsFederal,357471,2
1,Id,692571,92016
2,StatusName,692571,5
3,StatusCode,245633,1
4,TypeName,692571,3
...,...,...,...
133,StoppedSupplementIssueDate,9,9
134,OldRegNumber,5458,2104
135,OldSerialNumber,5444,9
136,OldFormNumber,5444,2339


In [215]:
edu_columns_df.to_csv('edu_columns.csv', sep = ',', index=False)

In [216]:
edu_df['StatusName'].unique()

array(['Недействующее', 'Действующее', 'Прекращено', 'Лишен аккредитации',
       'Приостановлено'], dtype=object)

In [217]:
edu_df['StatusName12'].unique()

array(['Недействующее', nan, 'Действующее', 'Прекращено',
       'Приостановлено в отношении отдельных уровней образования, укрупненных групп специальностей и направлений подготовки',
       'Возобновлено',
       'Лишение в отношении отдельных уровней образования, укрупненных групп специальностей и направлений подготовки',
       'Лишен аккредитации', 'Приостановлено'], dtype=object)

In [218]:
edu_df[(edu_df['StatusName'] != edu_df['StatusName12'])][['StatusName','StatusName12']]

,StatusName,StatusName12
3,Недействующее,NaN
6,Действующее,NaN
7,Действующее,NaN
10,Действующее,NaN
14,Недействующее,NaN
...,...,...
692524,Недействующее,Действующее
692525,Недействующее,NaN
692529,Действующее,NaN
692533,Действующее,NaN


In [238]:
columns = ['FullName','ShortName','PostAddress','ProgrammName','UGSName']
high_edu_df = edu_df[(
    (edu_df['EduLevelName'].str.contains('^(ВО -|Высшее) ', na=False, regex=True)) &
    (edu_df['StatusName12'].isin(['Действующее','Возобновлено'])))][columns]

/tmp/ipykernel_358639/3613693267.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (edu_df['EduLevelName'].str.contains('^(ВО -|Высшее) ', na=False, regex=True)) &


In [239]:
high_edu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34630 entries, 98 to 690816
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   FullName      34630 non-null  object
 1   ShortName     34481 non-null  object
 2   PostAddress   34626 non-null  object
 3   ProgrammName  34618 non-null  object
 4   UGSName       34618 non-null  object
dtypes: object(5)
memory usage: 1.6+ MB


In [240]:
high_edu_df.nunique()

FullName        1139
ShortName       1117
PostAddress     1151
ProgrammName     592
UGSName           92
dtype: int64

In [51]:
#.str.replace('\(.*\)','', regex=True).unique()

array([nan], dtype=object)

In [52]:
#high_edu_df['UGSName'] = high_edu_df['UGSName'].fillna(high_edu_df['ProgrammName'].str.replace('\(.*\)','', regex=True))

In [53]:
#high_edu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34630 entries, 98 to 690816
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   FullName      34630 non-null  object
 1   ShortName     34481 non-null  object
 2   PostAddress   34626 non-null  object
 3   ProgrammName  34618 non-null  object
 4   UGSName       34618 non-null  object
dtypes: object(5)
memory usage: 1.6+ MB


In [241]:
high_edu_df.dropna(subset = ['ProgrammName'], inplace=True)

In [242]:
high_edu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34618 entries, 98 to 690816
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   FullName      34618 non-null  object
 1   ShortName     34481 non-null  object
 2   PostAddress   34618 non-null  object
 3   ProgrammName  34618 non-null  object
 4   UGSName       34618 non-null  object
dtypes: object(5)
memory usage: 1.6+ MB


In [243]:
high_edu_df.nunique()

FullName        1134
ShortName       1117
PostAddress     1147
ProgrammName     592
UGSName           92
dtype: int64

In [244]:
high_edu_df['UGSName'].unique()

array(['Техника и технологии строительства',
       'Электроника, радиотехника и системы связи',
       'Электро- и теплоэнергетика',
       'Техника и технологии наземного транспорта',
       'Науки о здоровье и профилактическая медицина',
       'Биологические науки', 'Химические технологии',
       'Экономика и управление', 'История и археология',
       'Физико-технические науки и технологии',
       'Культуроведение и социокультурные проекты',
       'Политические науки и регионоведение',
       'Образование и педагогические науки', 'Психологические науки',
       'Информатика и вычислительная техника', 'Искусствознание',
       'Физическая культура и спорт', 'Управление в технических системах',
       'Информационная безопасность',
       'Средства массовой информации и информационно-библиотечное дело',
       'Языкознание и литературоведение',
       'Изобразительное и прикладные виды искусств',
       'Физика и астрономия', 'Технологии материалов',
       'Социология и социальн

In [245]:
for col in list(high_edu_df.columns):
    high_edu_df[col] = high_edu_df[col].str.replace(r'\s{2,}', ' ',regex=True).str.strip()

In [246]:
high_edu_df.nunique()

FullName        1134
ShortName       1117
PostAddress     1145
ProgrammName     592
UGSName           92
dtype: int64

In [247]:
high_edu_df.drop_duplicates(inplace=True)
high_edu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22069 entries, 98 to 690815
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   FullName      22069 non-null  object
 1   ShortName     21940 non-null  object
 2   PostAddress   22069 non-null  object
 3   ProgrammName  22069 non-null  object
 4   UGSName       22069 non-null  object
dtypes: object(5)
memory usage: 1.0+ MB


In [248]:
high_edu_df['FullName'].unique()

array(['федеральное государственное казенное военное образовательное учреждение высшего образования «Тюменское высшее военно-инженерное командное училище имени маршала инженерных войск А.И.Прошлякова» Министерства обороны Российской Федерации',
       'федеральное государственное бюджетное образовательное учреждение высшего образования «Череповецкий государственный университет»',
       'Федеральное государственное бюджетное образовательное учреждение дополнительного профессионального образования «Институт повышения квалификации Федерального медико-биологического агентства»',
       ...,
       'федеральное государственное автономное образовательное учреждение высшего образования «Национальный исследовательский университет «Московский институт электронной техники»',
       'Федеральное государственное бюджетное учреждение науки Институт физиологии растений им. К.А. Тимирязева Российской академии наук',
       'федеральное государственное бюджетное образовательное учреждение инклюзивног

In [249]:
maxColumnLenghts = []
for col in range(len(high_edu_df.columns)):
    maxColumnLenghts.append(max(high_edu_df.iloc[:,col].astype(str).apply(len)))
print('Max Column Lengths ', maxColumnLenghts)

Max Column Lengths  [297, 302, 151, 254, 77]


In [251]:
high_edu_df.reset_index(inplace=True, drop=True)

In [282]:
city_df = high_edu_df[['PostAddress']].drop_duplicates()
city_df['PostAddress'] = city_df['PostAddress'] + ","
city_df.reset_index(inplace=True, drop=True)
city_df

,PostAddress
0,"625001, г. Тюмень, улица Льва Толстого, д. 1,"
1,"162600, Вологодская область, г. Череповец, про..."
2,"125371, Российская Федерация, город Москва, Во..."
3,"460000, Российская Федерация, Оренбургская обл..."
4,"Республика Армения, г. Ереван, ул. Овсепа Эмин..."
...,...
1140,"628012, Тюменская область, Ханты-Мансийский ав..."
1141,"124498, г. Москва, г. Зеленоград, площадь Шоки..."
1142,"Российская Федерация, 127276, г. Москва, ул. Б..."
1143,"Российская Федерация, 357600, Ставропольский к..."


In [128]:
#city_df.drop(columns=['postcode','city','city1','city2','city3'], inplace=True)

In [283]:
city_df['postcode'] = city_df['PostAddress'].str.extract(r'(\d{6})', expand=True)

#city_df['city1'] = city_df['PostAddress'].str.replace(' ', '').str.extract(r'г\.(\S*?),')

city_df['city1'] = city_df['PostAddress'].str.replace('- ','-').str.replace(' -','-').str.replace(' Московской области', '').str.extract(r'г\.\s*(.*?),')
city_df['city2'] = city_df['PostAddress'].str.replace(' Челябинской области', '').str.extract(r'\Wгород\s+(.*?),')
city_df['city3'] = city_df['PostAddress'].str.extract(r'гор\.\s*(.*?),')
#city_df['city4'] = city_df['PostAddress'].str.extract(r'с\.\s*(.*?),')

city_df['city'] = city_df['city1']
city_df['city'] = city_df['city'].fillna(city_df['city2'])
city_df['city'] = city_df['city'].fillna(city_df['city3'])

#str.replace('\(.*\)','', regex=True))

In [ ]:
#city_df[(~city_df['city3'].isna())].head(30)

In [284]:
city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1145 entries, 0 to 1144
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PostAddress  1145 non-null   object
 1   postcode     1127 non-null   object
 2   city1        936 non-null    object
 3   city2        134 non-null    object
 4   city3        2 non-null      object
 5   city         1072 non-null   object
dtypes: object(6)
memory usage: 53.8+ KB


In [285]:
city_df.nunique()

PostAddress    1145
postcode        707
city1           153
city2            60
city3             2
city            163
dtype: int64

In [286]:
city_df[(~city_df['city'].isna())]['city'].unique()

array(['Тюмень', 'Череповец', 'Москва', 'Оренбург', 'Ереван',
       'Санкт-Петербург', 'Кемерово', 'Воронеж', 'Армавир', 'Казань',
       'Кострома', 'Набережные Челны', 'Рязань', 'Йошкар-Ола',
       'Ярославль', 'Киров', 'Архангельск', 'Красноярск', 'Ставрополь',
       'Владимир', 'Владивосток', 'Владикавказ', 'Белгород',
       'Железногорск', 'Брянск', 'Химки', 'Смоленск', 'Калининград',
       'Нижний Новгород', 'Ульяновск', 'Горно-Алтайск', 'Волгоград',
       'Верхняя Пышма', 'Елец', 'Майкоп', 'Альметьевск', 'Новосибирск',
       'Ростов-на-Дону', 'Уфа', 'Барнаул', 'Жуковский', 'Магадан',
       'Магнитогорск', 'Чебоксары', 'Пермь', 'Самара', 'Челябинск',
       'Липецк', 'Тамбов', 'Сургут', 'Томск', 'Екатеринбург', 'Улан-Удэ',
       'Иваново', 'Махачкала', 'Хабаровск', 'Краснодар', 'Пущино',
       'Глазов', 'Сочи', 'Астрахань', 'Чита', 'Грозный', 'Домодедово',
       'Южно-Сахалинск', 'Мурманск', 'Абакан', 'Тверь', 'Ижевск',
       'Симферополь', 'Благовещенск', 'Волжский',

In [260]:
# city_df[city_df['city2']=='ок']

,PostAddress,postcode,city1,city2,city3,city


In [181]:
# list(city_df[city_df['city2']=='ок']['PostAddress'])

[]

In [287]:
#list(city_df[(city_df['city1'].isna() & city_df['city2'].isna() & city_df['city3'].isna() & city_df['city4'].isna() & city_df['postcode'].isna())]['PostAddress'])
#list(city_df[(~city_df['city3'].isna())]['PostAddress'])
list(city_df[(city_df['city'].isna() & city_df['postcode'].isna())]['PostAddress'])


['Самарская область, Ставропольский район, Ставропольский лесхоз, Ягодинское лесничество, квартал № 5, оздоровительный комплекс «Алые паруса», корпус № 5,']

In [288]:
city_df.loc[city_df['PostAddress'].str.contains('Алые паруса'),'postcode'] = '445144'

In [289]:
city_df[city_df['PostAddress'].str.contains('Алые паруса')]
# 445144

,PostAddress,postcode,city1,city2,city3,city
988,"Самарская область, Ставропольский район, Ставр...",445144,NaN,NaN,NaN,NaN


In [293]:
city_df['PostAddress'] = city_df['PostAddress'].str.replace(r',$', '',regex=True)

In [294]:
city_df.set_index('PostAddress', inplace=True)
city_df.drop(columns=['city1','city2','city3'], inplace=True)

In [295]:
city_df

,postcode,city
PostAddress,,
"625001, г. Тюмень, улица Льва Толстого, д. 1",625001,Тюмень
"162600, Вологодская область, г. Череповец, проспект Луначарского, д. 5",162600,Череповец
"125371, Российская Федерация, город Москва, Волоколамское шоссе, д. 91",125371,Москва
"460000, Российская Федерация, Оренбургская область, г. Оренбург, ул. Советская/ ул. М. Горького/ пер. Дмитриевский, 6/45/7",460000,Оренбург
"Республика Армения, г. Ереван, ул. Овсепа Эмина, 123",NaN,Ереван
...,...,...
"628012, Тюменская область, Ханты-Мансийский автономный округ-Югра, г. Ханты-Мансийск, ул. Чехова, д. 16",628012,Ханты-Мансийск
"124498, г. Москва, г. Зеленоград, площадь Шокина, д. 1",124498,Москва
"Российская Федерация, 127276, г. Москва, ул. Ботаническая, д. 35",127276,Москва


In [298]:
high_edu_df = high_edu_df.join(city_df, on='PostAddress', how='left')
high_edu_df

,FullName,ShortName,PostAddress,ProgrammName,UGSName,postcode,city
0,федеральное государственное казенное военное о...,Тюменское высшее военно-инженерное командное у...,"625001, г. Тюмень, улица Льва Толстого, д. 1",Строительство уникальных зданий и сооружений,Техника и технологии строительства,625001,Тюмень
1,федеральное государственное казенное военное о...,Тюменское высшее военно-инженерное командное у...,"625001, г. Тюмень, улица Льва Толстого, д. 1",Специальные радиотехнические системы,"Электроника, радиотехника и системы связи",625001,Тюмень
2,федеральное государственное казенное военное о...,Тюменское высшее военно-инженерное командное у...,"625001, г. Тюмень, улица Льва Толстого, д. 1",Тепло- и электрообеспечение специальных технич...,Электро- и теплоэнергетика,625001,Тюмень
3,федеральное государственное казенное военное о...,Тюменское высшее военно-инженерное командное у...,"625001, г. Тюмень, улица Льва Толстого, д. 1",Транспортные средства специального назначения,Техника и технологии наземного транспорта,625001,Тюмень
4,федеральное государственное бюджетное образова...,Череповецкий государственный университет,"162600, Вологодская область, г. Череповец, про...",Общественное здравоохранение,Науки о здоровье и профилактическая медицина,162600,Череповец
...,...,...,...,...,...,...,...
22064,федеральное государственное бюджетное образова...,ФГБОУИ ВО «Московский государственный гуманита...,"107150, г. Москва, ул. Лосиноостровская, д. 49",Педагогическое образование,Образование и педагогические науки,107150,Москва
22065,федеральное государственное бюджетное образова...,ФГБОУИ ВО «Московский государственный гуманита...,"107150, г. Москва, ул. Лосиноостровская, д. 49",Психолого-педагогическое образование,Образование и педагогические науки,107150,Москва
22066,федеральное государственное бюджетное образова...,ФГБОУИ ВО «Московский государственный гуманита...,"107150, г. Москва, ул. Лосиноостровская, д. 49",Лингвистика,Языкознание и литературоведение,107150,Москва
22067,федеральное государственное бюджетное образова...,ФГБОУИ ВО «Московский государственный гуманита...,"107150, г. Москва, ул. Лосиноостровская, д. 49",Перевод и переводоведение,Языкознание и литературоведение,107150,Москва


In [299]:
high_edu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22069 entries, 0 to 22068
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   FullName      22069 non-null  object
 1   ShortName     21940 non-null  object
 2   PostAddress   22069 non-null  object
 3   ProgrammName  22069 non-null  object
 4   UGSName       22069 non-null  object
 5   postcode      21665 non-null  object
 6   city          21526 non-null  object
dtypes: object(7)
memory usage: 1.2+ MB


In [328]:
univer_df = high_edu_df[['FullName','postcode','city']].drop_duplicates()
univer_df.sort_values(by=['FullName','postcode','city'])

,FullName,postcode,city
5016,АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ВЫСШЕГО ...,115054,Москва
14032,Автономная некоммерческая образовательная орга...,394033,Воронеж
4048,Автономная некоммерческая образовательная орга...,690091,Владивосток
14070,Автономная некоммерческая образовательная орга...,191187,Санкт-Петербург
12599,Автономная некоммерческая образовательная орга...,394026,Воронеж
...,...,...,...
6936,частное образовательное учреждение высшего обр...,420111,Казань
10329,частное образовательное учреждение высшего обр...,190103,Санкт-Петербург
15273,частное образовательное учреждение высшего обр...,644010,Омск
19017,частное учреждение образовательная организация...,443001,Самара


In [329]:
univer_df = univer_df.groupby('FullName').agg(
    postcode=('postcode', 'last'),
    city=('city', 'last'))
univer_df

,postcode,city
FullName,,
АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ВЫСШЕГО ОБРАЗОВАНИЯ «НАЦИОНАЛЬНЫЙ ИНСТИТУТ ДИЗАЙНА»,115054,Москва
Автономная некоммерческая образовательная организация высшего образования «Воронежский экономико-правовой институт»,394033,Воронеж
Автономная некоммерческая образовательная организация высшего образования «Гуманитарный институт»,690091,Владивосток
Автономная некоммерческая образовательная организация высшего образования «Европейский университет в Санкт-Петербурге»,191187,Санкт-Петербург
Автономная некоммерческая образовательная организация высшего образования «Международный институт компьютерных технологий»,394026,Воронеж
...,...,...
частное образовательное учреждение высшего образования «Казанский инновационный университет имени В.Г. Тимирясова (ИЭУП)»,420111,Казань
частное образовательное учреждение высшего образования «Санкт-Петербургский университет технологий управления и экономики»,190103,Санкт-Петербург
частное образовательное учреждение высшего образования «Сибирский юридический университет»,644010,Омск


In [330]:
univer_df[univer_df['city'].isna()]

,postcode,city
FullName,,
Автономная некоммерческая образовательная организация высшего образования «Сколковский институт науки и технологий»,143025,None
Автономная некоммерческая организация высшего образования «Институт современного искусства»,121357,None
Автономная некоммерческая организация высшего образования «Международный банковский институт имени Анатолия Собчака»,191011,None
Автономная некоммерческая организация высшего образования «Национальный открытый институт г.Санкт-Петербург»,197183,None
Автономная некоммерческая организация высшего образования «Санкт-Петербургская юридическая академия»,192012,None
...,...,...
федеральное государственное бюджетное образовательное учреждение высшего образования «Санкт-Петербургский государственный институт культуры»,191186,None
федеральное государственное бюджетное образовательное учреждение высшего образования «Театральный институт имени Бориса Щукина при Государственном академическом театре имени Евгения Вахтангова»,119002,None
федеральное государственное бюджетное образовательное учреждение высшего образования «Уральская государственная консерватория имени М.П. Мусоргского»,620014,None


In [331]:
univer_df.reset_index(inplace=True)

In [333]:
univer_df[univer_df['FullName'].isin(['Частное образовательное учреждение высшего образования «Академия управления и производства»'])]
#univer_df[tuniver_dfmp['FullName'].isin(['Частное образовательное учреждение высшего образования «Русская христианская гуманитарная академия»'])]

,FullName,postcode,city
498,Частное образовательное учреждение высшего обр...,123007,Москва


In [317]:
#high_edu_df[high_edu_df['FullName'].isin(['Частное образовательное учреждение высшего образования «Академия управления и производства»'])]
high_edu_df[high_edu_df['FullName'].isin(['Частное образовательное учреждение высшего образования «Русская христианская гуманитарная академия»'])]

,FullName,ShortName,PostAddress,ProgrammName,UGSName,postcode,city
18094,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, Санкт-Петербург, наб. реки Фонтанки, д...",Психология,Психологические науки,191011,NaN
18095,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, Санкт-Петербург, наб. реки Фонтанки, д...",Филология,Языкознание и литературоведение,191011,NaN
18096,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, Санкт-Петербург, наб. реки Фонтанки, д...",Философия,"Философия, этика и религиоведение",191011,NaN
18097,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, Санкт-Петербург, наб. реки Фонтанки, д...",Религиоведение,"Философия, этика и религиоведение",191011,NaN
18098,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, Санкт-Петербург, наб. реки Фонтанки, д...",Искусства и гуманитарные науки,Искусствознание,191011,NaN
18099,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, Санкт-Петербург, наб. реки Фонтанки, д...","Философия, этика и религиоведение","Философия, этика и религиоведение",191011,NaN
18100,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, Санкт-Петербург, наб. реки Фонтанки, д...",Теология,Теология,191011,NaN
20184,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, г. Санкт-Петербург, набережная реки Фо...",Психология,Психологические науки,191011,Санкт-Петербург
20185,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, г. Санкт-Петербург, набережная реки Фо...",Филология,Языкознание и литературоведение,191011,Санкт-Петербург
20186,Частное образовательное учреждение высшего обр...,ЧОУ «Русская христианская гуманитарная академи...,"191011, г. Санкт-Петербург, набережная реки Фо...",Философия,"Философия, этика и религиоведение",191011,Санкт-Петербург


In [334]:
conn = psycopg2.connect(dbname="h_core", user="h_user", password="AUdZuMOGuP", host="84.201.178.7", port="55432")

In [71]:
with conn.cursor() as curs:
    for i in range(len(high_edu_df)):
        curs.execute('INSERT INTO h_raw_edu (Full_Name, Short_Name, Post_Address, Programm_Name, UGS_Name) VALUES (%s, %s, %s, %s, %s)',
                     (high_edu_df.loc[i,'FullName'],
                      high_edu_df.loc[i,'ShortName'],
                      high_edu_df.loc[i,'PostAddress'],
                      high_edu_df.loc[i,'ProgrammName'],
                      high_edu_df.loc[i,'UGSName']))
        if i % 1000 ==0:
            conn.commit()
            print(i)
    conn.commit()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000


In [336]:
conn.close()

# h_edu_directions

In [481]:
high_edu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22069 entries, 0 to 22068
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   FullName      22069 non-null  object
 1   ShortName     21940 non-null  object
 2   PostAddress   22069 non-null  object
 3   ProgrammName  22069 non-null  object
 4   UGSName       22069 non-null  object
 5   postcode      21665 non-null  object
 6   city          21526 non-null  object
dtypes: object(7)
memory usage: 1.2+ MB


In [484]:
directions = list(high_edu_df['UGSName'].unique())

In [489]:
len(directions)

92

In [3]:
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [486]:
embeddings = embedding_model.encode(directions)

In [488]:
embeddings.shape

(92, 384)

In [501]:
#str(embeddings[0])
#','.join(embeddings[0])
vec = '[' + (','.join(map(str, embeddings[0]))) + ']'
vec

'[0.014818062,0.15507291,-0.0496795,-0.01601848,-0.13945751,-0.21684112,-0.21002753,0.18450728,-0.1822346,0.064020306,-0.12507756,0.04620213,0.25460446,-0.19469808,-0.0021316165,0.30306473,-0.08526663,0.051373906,0.15184297,-0.14099512,0.23829055,-0.45287052,0.082919225,-0.36378178,0.21015082,0.08741941,-0.0767306,-0.028509496,0.2873407,-0.2600881,-0.1554396,-0.040924422,0.21924493,0.050886218,0.04712463,0.33915973,0.37105805,-0.0949441,-0.19545816,-0.062854275,-0.2699005,-0.16401184,0.12049014,0.10212083,0.06053189,0.29815546,0.059050467,-0.15230922,-0.19088647,-0.1798452,0.080022775,-0.2019277,-0.080322124,-0.3240916,0.07816026,0.021856574,0.49283442,0.25578073,-0.004271287,-0.07984231,0.29070956,0.012343615,-0.5494548,0.2906653,0.3294614,0.001795475,-0.02161127,0.31317583,-0.10844878,0.13917291,-0.058879852,-0.03065276,-0.12646084,-0.15892366,0.06314466,-0.035408568,0.065291785,0.22263972,0.1439357,-0.14159547,0.15646899,-0.012380638,-0.034086943,0.11248577,0.041984957,0.2518981,-0.

In [506]:
conn = psycopg2.connect(dbname="h_core", user="h_user", password="AUdZuMOGuP", host="84.201.178.7", port="55432")

In [507]:
with conn.cursor() as curs:
    for i in range(len(directions)):
        curs.execute('INSERT INTO h_edu_direction (value, embedding) VALUES (%s, %s)', (directions[i], '[' + (','.join(map(str, embeddings[i]))) + ']'))
    conn.commit()

In [508]:
conn.close()

# coordinates by city

In [339]:
towns_df = pd.read_csv('https://raw.githubusercontent.com/epogrebnyak/ru-cities/main/assets/towns.csv', delimiter = ',', usecols=["city", "lat", "lon"])
towns_df

,city,lat,lon
0,Алейск,52.492251,82.779361
1,Барнаул,53.347997,83.779806
2,Белокуриха,51.996152,84.983960
3,Бийск,52.539386,85.213845
4,Горняк,50.997962,81.464336
...,...,...,...
1112,Ростов,57.205132,39.437862
1113,Рыбинск,58.048550,38.858412
1114,Тутаев,57.867499,39.536963
1115,Углич,57.522425,38.302004


In [340]:
univer_df['city'].nunique()

163

In [343]:
cities = list(univer_df['city'].unique())

In [344]:
towns_df[towns_df['city'].isin(cities)].count()

city    157
lat     157
lon     157
dtype: int64

In [417]:
coords4towns_df = towns_df[towns_df['city'].isin(cities)]

In [418]:
coords4towns_df.rename(columns={'lat':'latitude','lon':'longitude'}, inplace=True)
coords4towns_df.set_index('city', inplace=True)

/tmp/ipykernel_358639/2761677130.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coords4towns_df.rename(columns={'lat':'latitude','lon':'longitude'}, inplace=True)


In [419]:
coords4towns_df

,latitude,longitude
city,,
Барнаул,53.347997,83.779806
Бийск,52.539386,85.213845
Благовещенск,50.290659,127.527198
Архангельск,64.539429,40.516961
Астрахань,46.365565,48.055924
...,...,...
Челябинск,55.160262,61.400808
Грозный,43.318014,45.698291
Чебоксары,56.143830,47.248978


In [422]:
univer_df = univer_df.join(coords4towns_df, on='city', how='left')

In [423]:
univer_df

,FullName,postcode,city,latitude,longitude
0,АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ВЫСШЕГО ...,115054,Москва,55.754047,37.620405
1,Автономная некоммерческая образовательная орга...,394033,Воронеж,51.659333,39.196923
2,Автономная некоммерческая образовательная орга...,690091,Владивосток,43.116490,131.882394
3,Автономная некоммерческая образовательная орга...,191187,Санкт-Петербург,59.939131,30.315900
4,Автономная некоммерческая образовательная орга...,394026,Воронеж,51.659333,39.196923
...,...,...,...,...,...
1129,частное образовательное учреждение высшего обр...,420111,Казань,55.794358,49.111497
1130,частное образовательное учреждение высшего обр...,190103,Санкт-Петербург,59.939131,30.315900
1131,частное образовательное учреждение высшего обр...,644010,Омск,54.984857,73.367452
1132,частное учреждение образовательная организация...,443001,Самара,53.195031,50.106952


In [424]:
univer_df[univer_df['latitude'].isna()]

,FullName,postcode,city,latitude,longitude
6,Автономная некоммерческая образовательная орга...,143025,None,NaN,NaN
19,Автономная некоммерческая организация высшего ...,121357,None,NaN,NaN
23,Автономная некоммерческая организация высшего ...,191011,None,NaN,NaN
31,Автономная некоммерческая организация высшего ...,197183,None,NaN,NaN
35,Автономная некоммерческая организация высшего ...,192012,None,NaN,NaN
...,...,...,...,...,...
1004,федеральное государственное бюджетное учрежден...,196603,Пушкин,NaN,NaN
1035,федеральное государственное бюджетное учрежден...,600901,None,NaN,NaN
1038,федеральное государственное бюджетное учрежден...,194021,None,NaN,NaN
1042,федеральное государственное казенное военное о...,302034,Орёл,NaN,NaN


In [425]:
univer_df[univer_df['latitude'].isna()]['postcode'].unique()

array(['143025', '121357', '191011', '197183', '192012', None, '196605',
       '125499', '192242', '192238', '301000', '630559', '142279',
       '664520', '141142', '143050', '140143', '397463', '346735',
       '601125', '198035', '140032', '190121', '117279', '678671',
       '194044', '192019', '123007', '117036', '191186', '119017',
       '620219', '143030', '119991', '152020', '199034', '620014',
       '117342', '125412', '644033', '369167', '630090', '196158',
       '197110', '191025', '109439', '191002', '720000', '212005',
       '734025', '400066', '445144', '111024', '194064', '141435',
       '141261', '308503', '243365', '140155', '346493', '664038',
       '156530', '641300', '302019', '302028', '121165', '191028',
       '141221', '196601', '119002', '196603', '600901', '194021',
       '302034', '640016'], dtype=object)

# coordinates by postcode

In [359]:
POST_SERVICE_URL = 'https://почта.ru.com/{}'

In [426]:
#post_codes = list(univer_df['postcode'].unique())[:2]
#post_codes = list(univer_df['postcode'].unique())
post_codes = list(univer_df[univer_df['latitude'].isna()]['postcode'].unique())

len(post_codes)

75

In [427]:
post_codes[0]

'143025'

In [428]:
print(POST_SERVICE_URL.format(post_codes[0]))

https://почта.ru.com/143025


In [429]:
delay_ = 6
post_codes_df = pd.DataFrame(columns = ['postcode','latitude', 'longitude'])
for cnt, item in enumerate(post_codes):
    print('cnt=', cnt)
    while 1:
        time.sleep(delay_)
        response = requests.request('GET',POST_SERVICE_URL.format(item))
        resp_html = response.text
        if response.status_code == 429:
            delay_ += 1
            print('cnt=', cnt, ' new delay=', delay_)
        else:
            break
    if response.status_code == 200:
        coords_txt = re.search('maps-yandex\?(\S*?)"', resp_html).group(0)[12:][:-1].split('&')
        coords = [x.split('=')[1] for x in coords_txt] # широта, долгота
        post_codes_df.loc[len(post_codes_df)] = [item, coords[0], coords[1]]
    else:
        print('cnt=', cnt, ' postcode=', item,' delay=', delay_)

cnt= 0
cnt= 1
cnt= 2
cnt= 3
cnt= 4
cnt= 5
cnt= 5  postcode= None  delay= 6
cnt= 6
cnt= 7
cnt= 8
cnt= 9
cnt= 10
cnt= 11
cnt= 12
cnt= 13
cnt= 14
cnt= 15
cnt= 16
cnt= 17
cnt= 18
cnt= 19
cnt= 20
cnt= 21
cnt= 22
cnt= 23
cnt= 24
cnt= 25
cnt= 26
cnt= 27
cnt= 28
cnt= 29
cnt= 30
cnt= 31
cnt= 31  postcode= 620219  delay= 6
cnt= 32
cnt= 33
cnt= 34
cnt= 35
cnt= 36
cnt= 37
cnt= 38
cnt= 39
cnt= 40
cnt= 41
cnt= 42
cnt= 43
cnt= 44
cnt= 45
cnt= 46
cnt= 47
cnt= 47  postcode= 720000  delay= 6
cnt= 48
cnt= 48  postcode= 212005  delay= 6
cnt= 49
cnt= 49  postcode= 734025  delay= 6
cnt= 50
cnt= 51
cnt= 52
cnt= 53
cnt= 54
cnt= 55
cnt= 56
cnt= 57
cnt= 58
cnt= 59
cnt= 60
cnt= 61
cnt= 62
cnt= 63
cnt= 64
cnt= 65
cnt= 66
cnt= 67
cnt= 68
cnt= 69
cnt= 70
cnt= 71
cnt= 72
cnt= 73
cnt= 74


In [431]:
post_codes_df.set_index('postcode', inplace=True)

In [435]:
univer_df = univer_df.join(post_codes_df, on='postcode', how='left', rsuffix='_1')

In [436]:
univer_df

,FullName,postcode,city,latitude,longitude,latitude_1,longitude_1
0,АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ВЫСШЕГО ...,115054,Москва,55.754047,37.620405,NaN,NaN
1,Автономная некоммерческая образовательная орга...,394033,Воронеж,51.659333,39.196923,NaN,NaN
2,Автономная некоммерческая образовательная орга...,690091,Владивосток,43.116490,131.882394,NaN,NaN
3,Автономная некоммерческая образовательная орга...,191187,Санкт-Петербург,59.939131,30.315900,NaN,NaN
4,Автономная некоммерческая образовательная орга...,394026,Воронеж,51.659333,39.196923,NaN,NaN
...,...,...,...,...,...,...,...
1129,частное образовательное учреждение высшего обр...,420111,Казань,55.794358,49.111497,NaN,NaN
1130,частное образовательное учреждение высшего обр...,190103,Санкт-Петербург,59.939131,30.315900,NaN,NaN
1131,частное образовательное учреждение высшего обр...,644010,Омск,54.984857,73.367452,NaN,NaN
1132,частное учреждение образовательная организация...,443001,Самара,53.195031,50.106952,NaN,NaN


In [442]:
univer_df = univer_df.astype({'latitude_1':'float','longitude_1':'float'})

In [447]:
univer_df['latitude'] = univer_df['latitude'].fillna(univer_df['latitude_1'])
univer_df['longitude'] = univer_df['longitude'].fillna(univer_df['longitude_1'])

In [452]:
univer_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1146 entries, 0 to 1133
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   FullName   1146 non-null   object 
 1   postcode   1129 non-null   object 
 2   city       1074 non-null   object 
 3   latitude   1136 non-null   float64
 4   longitude  1136 non-null   float64
dtypes: float64(2), object(3)
memory usage: 53.7+ KB


In [451]:
univer_df.drop(columns=['latitude_1','longitude_1'], inplace=True)

In [460]:
univer_df.reset_index(inplace=True, drop=True)

In [461]:
univer_df[univer_df['latitude'].isna()]

,FullName,postcode,city,latitude,longitude


In [454]:
univer_df.dropna(subset = ['latitude'], inplace=True)

In [476]:
conn = psycopg2.connect(dbname="h_core", user="h_user", password="AUdZuMOGuP", host="84.201.178.7", port="55432")

In [469]:
with conn.cursor() as curs:
    for i in range(len(univer_df)):
        curs.execute('INSERT INTO h_edu_universities (full_name, postcode, city, latitude, longitude) VALUES (%s, %s, %s, %s, %s)',
                     (univer_df.loc[i,'FullName'],
                      univer_df.loc[i,'postcode'],
                      univer_df.loc[i,'city'],
                      univer_df.loc[i,'latitude'],
                      univer_df.loc[i,'longitude']))
        if i % 1000 ==0:
            conn.commit()
            print(i)
    conn.commit()

0
1000


In [477]:
with conn.cursor() as curs:
    curs.execute('insert into h_edu_direct2univ (direction_id, university_id) ' +
                 'select distinct d.direction_id, u.university_id ' +
                 'from h_raw_edu r, h_edu_direction d, h_edu_universities u ' +
                 'where d.value = r.ugs_name and u.full_name = r.full_name')
conn.commit()

In [478]:
conn.close()

# test selects with embedding conditions

In [4]:
vk_faculty = 'Факультет биоинженерии и биоинформатики'

coords = (53.507852, 49.420411)

In [ ]:
select_query = 'select * from (select embedding <-> %s as cos, d.direction_id, d.value from h_edu_direction d ORDER BY d.embedding <-> %s limit 2) d'

In [571]:
select_query = ('select * from ' + 
                ' (select cos, faculty, full_name, city, dist, ' +
                '     row_number() over (partition by faculty order by dist) univ_dist_rank ' +
                '     from ' +
                '   (select distinct d.cos, d.faculty, u.full_name, u.city, ' +
                '     get_haversine_distance(u.latitude, u.longitude, %s, %s) dist ' +
                '    from ' +
                '     (select d.direction_id, d.value as faculty, '
                '       embedding <-> %s as cos '
                '      from h_edu_direction d order by d.embedding <-> %s limit 2) d, ' +
                '     h_edu_direct2univ d2u, h_edu_universities u ' +
                '    where d2u.direction_id = d.direction_id and u.university_id = d2u.university_id) u) u ' +
                'where univ_dist_rank < 10 ' +
                'order by faculty, univ_dist_rank, full_name')

In [6]:
embeddings4select = embedding_model.encode(vk_faculty)

In [7]:
embeddings4select = '[' + (','.join(map(str, embeddings4select))) + ']'

In [9]:
conn = psycopg2.connect(dbname="h_core", user="h_user", password="AUdZuMOGuP", host="84.201.178.7", port="55432")

In [575]:
with conn.cursor() as curs:
    curs.execute(select_query,(coords[0], coords[1], embeddings4select,embeddings4select))
    rs = curs.fetchall()

In [12]:
select_query = 'select embedding <-> %s as cos, * from h_additional_data order by embedding <-> %s limit 9'

In [13]:
with conn.cursor() as curs:
    curs.execute(select_query,(embeddings4select,embeddings4select))
    rs = curs.fetchall()

In [14]:
rs

[(0.7847840295117406,
  '3ad7e6fb-7a43-4b20-9662-fe5721d43542',
  datetime.datetime(2023, 12, 10, 9, 29, 54, 966043),
  datetime.datetime(2023, 12, 10, 9, 29, 54, 966045),
  'be05f3b1-83ad-431a-89a2-027b002a2a10',
  '0707cbb7-bd81-4d68-89b6-f976c6b18d97',
  '[0.049131718,0.35103276,-0.028960692,-0.0022693933,-0.13416173,-0.2506562,-0.17928338,0.29163027,-0.058073618,0.22603568,0.11569243,-0.46069965,-0.19010292,0.29466888,-0.16147867,-0.14397892,-0.1609763,0.05348116,0.22194193,-0.046665143,0.19036303,0.097731374,0.27300388,-0.050807204,-0.044407174,-0.040840067,-0.32035193,-0.09240401,-0.050059866,-0.22090918,0.21913697,0.023545286,0.21290702,-0.006368208,-0.023853099,0.29873568,-0.15738061,0.14523008,0.0319321,0.2660536,-0.042062473,-0.15821715,0.13299641,0.20994815,-0.058819022,-0.03333393,0.00718725,-0.14683689,-0.015347939,-0.43625528,-0.09787679,-0.26458797,-0.1789081,-0.21198282,0.12159367,0.0038452053,0.22935809,-0.121762566,-0.11999014,-0.023482295,0.2787278,0.11213771,-0.2844

In [566]:
conn.close()